<a href="https://colab.research.google.com/github/MCanela-1954/DataSci_Course/blob/main/%5BDATA-05E%5D%20Example%20-%20Bay%20Wheels%20operations%20data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [DATA-05E] Example - Bay Wheels operations data

## Introduction

**Bike sharing systems** are a means of renting bicycles where the process of obtaining membership, rental, and bike return is automated via a network of kiosk locations throughout an urban area. Using these systems, people are able to rent a bike from one location and return it to a different place on an as-needed basis. According to PBSC Urban Solutions, there were, as of August 2021, nearly 10 million shared bikes and 3,000 bike sharing systems across the world.

There is a great interest in these systems due to their important role in traffic, environmental and health issues. Apart from interesting real world applications of bike sharing systems, the characteristics of the data generated by these systems makes them attractive for researchers. Opposed to other transport services such as bus or subway, the duration of travel, departure location, arrival location and time elapsed is explicitly recorded. This feature turns a bike sharing system into a virtual sensor network, which can be used for studying mobility within the area. Hence, it is expected that most of important events in the area could be detected via monitoring these data.

This example uses two years of data from the **Bay Wheels Bike Share Program**, which brings point-to-point bike sharing to Bay Area cities, serving Berkeley, Emeryville, Oakland, San Jose and San Francisco. Bay Wheels is a partnership between MTC, the five local governments, and Motivate (a subsidiary of Lyft).  

The Bay Wheels Program has undergone a bit of change. It started as the Bay Area Bike Share in August 2013. On June 28, 2017, the system was officially re-launched as Ford GoBike in a partnership with Ford Motor Company. After Motivate's acquisition by Lyft, the system was renamed to Bay Wheels on June 11, 2019. In 2018, Ford GoBike added **electric bicycles** and **dockless bike share**. Nowadays, electric bicycles account for two thirds of the rides. Classic bikes are docked, so they are picked at one station and left at another station (or at the same one). E-bikes can use the **docking stations** or can be locked to a **city bike rack**.

## The data set

The data for the example come in two tables. The table `bay_stations` (a CSV file) contains data on 642 docking stations. These stations are not the same along the three-year period, since the organization is dynamic. The columns are:

* `station_id`, a unique identifier of the station. The first two characters indicate the location, with values 'BK' (Berkeley), 'SF' (San Francisco), 'SJ' (San Jose) and 'OK' (Oakland).

* `station_name`, the name of the station, referred to its location.

* `station_latitude`, the latitude of the station, with three decimals.

* `station_longitude`, the longitude of the station, with three decimals.

The table `bay_rides` (five zipped CSV files) contains information on all rides starting in the years 2021, 2022 and 2023, a total of 7,162,392 rides. The columns are:

* `user_type`, either 'casual' or 'member'.

* `bike_type`, either 'classic' or 'electric'.

* `start_time`, when the bike was picked, as yyyy-mm-dd hh:mm:ss.

* `start_station_id`, the identifier of the docking station where the ride started, missing when no station was involved.

* `end_time`, when the bike was returned, as yyyy-mm-dd hh:mm:ss.

* `end_station_id`, the identifier of the docking station where the ride ended, missing when no station was involved.

## Questions

Q1. Add a new column, `hour`, to the table `rides`, containing the hour of the start time, in `datetime64` format. Example: the hour for `2021-01-01 01:20:23` will be `2021-01-01 01:00:00`.

Q2. **Group by** `hour` and **aggregate** the data so that you get a a new table with two columns, `casual` and `member`, containing, for every hour, the total number of rides of the types of users.

Q3. After aggregating the data in the preceding question, can see you see a **time trend** in the number of rides? To visualize the trend, would it be better to aggregate more, *e.g*. to use daily data? Do you see a similar trend for the two user types?

Q4. Can you describe in an easy way the pattern for **intraday variation** (across hours) of the number of rides? Is this pattern different for the two user types?

Q5. Same questions for **intraweek variation** (across weekdays).

Q6. What about **monthly seasonality**?

## Importing the data

We import Pandas as usual.

In [ ]:
import pandas as pd

The table `rides` has been split in five data sets stored in zipped CSV files in GitHub, with a common path. So, we create a variable containing that path.

In [ ]:
path = 'https://raw.githubusercontent.com/MCanela-1954/Data/main/'

Next, we import the data from every files to a data frame.

In [ ]:
rides1 = pd.read_csv(path + 'bay_rides-1.csv.zip')
rides2 = pd.read_csv(path + 'bay_rides-2.csv.zip')
rides3 = pd.read_csv(path + 'bay_rides-3.csv.zip')
rides4 = pd.read_csv(path + 'bay_rides-4.csv.zip')
rides5 = pd.read_csv(path + 'bay_rides-5.csv.zip')

With the Pandas function `concat()`, we can get the **union** of these five data sets as a single data frame.

In [ ]:
rides = pd.concat([rides1, rides2, rides3, rides4, rides5])

## Exploring the data

We check the content of the data frame `rides` as in other examples, with the methods `.info()` and `.head()`. Everything is as expected. Note that the station ID is missing for some of the electric bikes, as explained in the introduction.

In [ ]:
rides.info()

In [ ]:
rides.head()

## Q1. Add a column with the hour

The start and end times have type `str`, so we can create the new column by means of string methods. There many ways to do it. A simple one is to drop the last six characters, which are the minutes and seconds, appending instead the string `':00:00'`.

In [ ]:
rides['hour'] = rides['start_time'].str[:-6] + ':00:00'
rides.head()

We need to convert this new column to a datetime type, in order to be able to extract the weekdays. **Type conversions** in Pandas are easily managed with the method `.astype()`.

In [ ]:
rides['hour'] = rides['hour'].astype('datetime64[ns]')
rides.info()

## Q2. Aggregate to hourly data

To build the data set for this question we need two additional columns, `casual` and `member`. We create them as dummies, so that we can aggregate them in order to get the number of rides for each group.

In [ ]:
rides['casual'] = (rides['user_type'] == 'casual')
rides['member'] = (rides['user_type'] == 'member')

Now, we group by hour and aggregate with the function `sum()`. To get a cleaner picture, we include only the columns that are relevant for the rest of this example. Note the double bracketing, which is needed, since the columns included have to be specified as a list.

In [ ]:
df = rides[['hour', 'casual', 'member']].groupby(by='hour').sum()
df.head()

Note that `hour` is no longer a column, but the index. This is the default of the method `.groupby()`. Because of the type conversion performed above, to data type `datetime64`, this index is a `DatetimeIndex`.

In [ ]:
df.index

## Q3. Time trend

We get rid of the index name, so that it will not appear in the plots below, which might be confusing.

In [ ]:
df.index.name = None

For clarity, we create a new column with the total number of rides.

In [ ]:
df['total'] = df['member'] + df['casual']

Time trends are typically spotted by means of **line charts**. See below a sample using the hourly total number of rides.

In [ ]:
df['total'].plot(figsize=(8,5), title='Figure 1. Hourly total demand', color='black', linewidth=1);

We see here a combination of a trend with time-based patterns, but it is difficult to conclude much with so many observations and the current granularity of the data. Since intraday patterns can be responsible for a significant part of the variation that we see in the chart, we aggregate to a daily data set. We use the mean so the vertical scale in the successive charts remains the same. Note that, here, we don't use `.groupby()`, but `.resample()`, and we don't need to create a new data set for plotting.

In [ ]:
df['total'].resample('D').mean().plot(figsize=(8,5), title='Figure 2. Daily total demand',
	color='black', linewidth=1);

Now, the picture is more clear, though part of the variation is probably due to weekends and holidays. Going a bit further, we can aggregate to a weekly data set, again with `.resample()`.

In [ ]:
df['total'].resample('W').mean().plot(figsize=(8,5), title='Figure 3. Weekly total demand',
	color='black', linewidth=1);

We can aggregate the data even further, to a monthly data set. The new chart shows a combination of a trend plus monthly seasonality.

In [ ]:
df['total'].resample('ME').mean().plot(figsize=(8,5), title='Figure 4. Monthly total demand',
	color='black', linewidth=1);

To see whether the trend is the same for the two user types, we can draw separate charts, which show that the trend only happens in the member group. This may be due to workforce discontinuing remote work and going back to office after the Stay Home order.

In [ ]:
df['casual'].resample('ME').mean().plot(figsize=(8,5), title="Figure 5. Casuals' monthly total demand",
	color='black', linewidth=1);

In [ ]:
df['member'].resample('ME').mean().plot(figsize=(8,5), title="Figure 6. Members' monthly total demand",
	color='black', linewidth=1);

## Q4. Intraday variation

To examine the intraday variation, we need to extract an average value for every hour. First, we create a column containing (only) the hour, as an integer, with the method `.hour`.

In [ ]:
df['hour'] = df.index.hour
df.head()

Now, we group by hour and aggregate to 24 average values. For the casual users, the aggregate data look like:

In [ ]:
print(df[['casual', 'hour']].groupby('hour').mean().round(1))

You may prefere to see this in a **bar chart**. Comparing the charts of the two user types, we see the influence of work schedule in the intraday variation pattern of the members.  

In [ ]:
df[['casual', 'hour']].groupby('hour').mean().plot.bar(figsize=(7,5),
	title="Figure 7. Intraday variation of casuals' average demand", color='gray', legend=False);

In [ ]:
df[['member', 'hour']].groupby('hour').mean().plot.bar(figsize=(7,5),
	title="Figure 8. Intraday variation of members' average demand", color='gray', legend=False);

Or you may prefer to pack both charts as a **stacked bar chart**, which is easy in Pandas, as we see next.

In [ ]:
df[['casual', 'member', 'hour']].groupby('hour').mean().plot.bar(figsize=(7,5),
	title='Figure 9. Intraday variation of average demand', color=['0.4', '0.7'], stacked=True);

## Q5. Intraweek variation

A similar approach can be followed for the intraweek variation. First, we create a new column by extracting the weekday from the index, now with `.weekday` (Monday = 0, Sunday = 6).

In [ ]:
df['weekday'] = df.index.weekday

The stacked bar chart is obtained as for question Q4. As for the intraday variation, the different patterns visualized suggest that members and casuals use the bike  travelling with different purposes (work/leisure).

In [ ]:
df[['casual', 'member', 'weekday']].groupby('weekday').mean().plot.bar(figsize=(7,5),
	title= 'Figure 10. Intraweek variation of total demand', color=['0.4', '0.7'], stacked=True);

## Q6. Monthly seasonality

We can visualize the monthly seasonality by means of twelve monthly averages. Again, we create a new column, now with months (January = 1, December = 12).

In [ ]:
df['month'] = df.index.month
df[['casual', 'member', 'month']].groupby('month').mean().round(1)

We can plot just plot the total demand here, since the seasonal patterns are about the same for casuals and members.

In [ ]:
df[['total', 'month']].groupby('month').mean().plot(figsize=(8,5),
    title='Figure 11. Monthly seasonality', color='black', linewidth=1, legend=False);

## Homework

1. Perform an analysis of the demand in which the patterns of variation for the demand of electric and classic bikes are compared. Are classic bikes lagging behind electric bikes?

2. Which are the top-10 starting stations? Are they the same as the top-10 ending stations?

3. How frequent are circular rides, starting and ending at the same station?

4. Are there stations with very low activity, so you can consider dropping them?

5. Seasonal patterns can be different across the stations of the Bay Wheels network. How can you detect the stations where the between-month variation is highest?
